In [1]:
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold,train_test_split,KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Models
from xgboost import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier

import os
import pandas as pd
import numpy as np
import time


import warnings
warnings.filterwarnings("ignore")


from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


In [2]:
sample_submission = pd.read_csv("/kaggle/input/playground-series-s3e18/sample_submission.csv")
test_data = pd.read_csv("/kaggle/input/playground-series-s3e18/test.csv")
train_data = pd.read_csv("/kaggle/input/playground-series-s3e18/train.csv")

In [3]:
train_data = train_data.drop(columns=['EC3', 'EC4', 'EC5', 'EC6'], axis=1)

In [4]:
y1  = train_data[['EC1']]
y2  = train_data[['EC2']]

X  = train_data.drop(columns=['EC1', 'EC2'], axis=1)

xgb = XGBClassifier()

estimators = [
    ('xgb',XGBClassifier()),
    ('rf', RandomForestClassifier(n_estimators=1000, random_state=2)),
    ('svr',make_pipeline(StandardScaler(),LinearSVC(random_state=2,max_iter=500)))
]

clf = StackingClassifier(
    estimators=estimators, final_estimator=KNeighborsClassifier(n_neighbors=75)
)

oof_preds1 = np.zeros(y1.shape)
oof_preds2 = np.zeros(y2.shape)


test_preds1 = np.zeros((test_data.shape[0], y1.shape[1]))
test_preds2 = np.zeros((test_data.shape[0], y2.shape[1]))


oof_scores1 = []
oof_scores2 = []


kf = KFold(n_splits = 5)

In [5]:
test_preds1.shape

(9893, 1)

### *EC 1*

In [6]:
data = kf.split(X,y1)

for i, (train_index, val_index) in enumerate(data):
    
    
    x_train_kf, x_val_kf = X.iloc[train_index], X.iloc[val_index]
    y_train_kf, y_val_kf = y1.iloc[train_index], y1.iloc[val_index]
    
    clf.fit(x_train_kf,y_train_kf)
    
    val_preds = clf.predict_proba(x_val_kf)
    val_preds = np.array(val_preds)[:,1]
    val_preds = val_preds.reshape(val_preds.shape[0],1)
    oof_preds1[val_index] = val_preds
    
    score = roc_auc_score(np.ravel(y_val_kf), np.ravel(val_preds))
    oof_scores1.append(score)
    preds = clf.predict_proba(test_data)
    preds = np.array(preds)[:,1] 
    preds = preds.reshape(preds.shape[0],1)
    test_preds1 += preds / 5
    
    print(i , "Score = ",  score)
    
    
print("Mean Score: ",np.mean(oof_scores1))


0 Score =  0.6918504566141294
1 Score =  0.6921225057719262
2 Score =  0.6885828087857363
3 Score =  0.6948607635276364
4 Score =  0.6606501662167631
Mean Score:  0.6856133401832383


## *EC 2*

In [7]:
data = kf.split(X,y2)
for i, (train_index, val_index) in enumerate(data):
    x_train_kf, x_val_kf = X.iloc[train_index], X.iloc[val_index]
    y_train_kf, y_val_kf = y2.iloc[train_index], y2.iloc[val_index]
    clf.fit(x_train_kf,y_train_kf)
    
    val_preds = clf.predict_proba(x_val_kf)
    val_preds = np.array(val_preds)[:,1]
    val_preds = val_preds.reshape(val_preds.shape[0],1)
    oof_preds2[val_index] = val_preds
    
    score = roc_auc_score(np.ravel(y_val_kf), np.ravel(val_preds))
    oof_scores2.append(score)
    preds = clf.predict_proba(test_data)
    preds = np.array(preds)[:,1] 
    preds = preds.reshape(preds.shape[0],1)
    test_preds2 += preds / 5
    
    print(i , "Score = ",  score)
    
    
print("Mean Score: ",np.mean(oof_scores2))

0 Score =  0.5781758827108541
1 Score =  0.559884936701496
2 Score =  0.5677994791666667
3 Score =  0.5626183127572016
4 Score =  0.550097928721635
Mean Score:  0.5637153080115708


In [8]:
submission = sample_submission
submission['EC1'] = test_preds1
submission['EC2'] = test_preds2

In [9]:
submission

,id,EC1,EC2
0,14838,0.408000,0.757333
1,14839,0.837333,0.749333
2,14840,0.680000,0.728000
3,14841,0.781333,0.808000
4,14842,0.789333,0.768000
...,...,...,...
9888,24726,0.642667,0.760000
9889,24727,0.802667,0.824000
9890,24728,0.485333,0.829333
9891,24729,0.512000,0.818667


In [10]:
submission.to_csv("submission.csv",index=False)